In [59]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

In [60]:
data = pickle.load( open( "/content/drive/MyDrive/Colab Notebooks/data_simple.p", "rb" ) )
data.head()

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,diag_1_Circulatory,diag_1_Diabetes,diag_1_Digestive,diag_1_Genitourinary,diag_1_Injury,diag_1_Musculoskeletal,diag_1_Neoplasms,diag_1_Other,diag_1_Respiratory,diag_2_Circulatory,diag_2_Diabetes,diag_2_Digestive,diag_2_Genitourinary,diag_2_Injury,diag_2_Missing,diag_2_Musculoskeletal,diag_2_Neoplasms,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Genitourinary,diag_3_Injury,diag_3_Missing,diag_3_Musculoskeletal,diag_3_Neoplasms,diag_3_Other,diag_3_Respiratory,admission_type_Elective,admission_type_Emergency,admission_type_NaN,admission_type_Newborn,admission_type_Trauma Center,admission_type_Urgent,metformin_bool,repaglinide_bool,nateglinide_bool,chlorpropamide_bool,glimepiride_bool,acetohexamide_bool,glipizide_bool,glyburide_bool,tolbutamide_bool,pioglitazone_bool,rosiglitazone_bool,acarbose_bool,miglitol_bool,troglitazone_bool,tolazamide_bool,insulin_bool,glyburide-metformin_bool,glipizide-metformin_bool,glimepiride-pioglitazone_bool,metformin-rosiglitazone_bool,metformin-pioglitazone_bool,age_label
0,8222157,1,6,25,1,1,41,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,720936,1,6,25,1,10,56,2,24,0,0,0,9,0,2,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7
2,661014,0,6,25,7,9,61,4,25,0,0,0,9,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6
3,1719279,0,6,25,7,11,68,0,25,0,0,0,9,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4
4,7919802,1,6,25,7,6,47,2,13,0,0,0,8,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [61]:
pd.set_option('display.max_columns', None)

# Training classification models
The goal is to determine if a patient will be readmitted within 30 days. To this end we want to train classification models. We will select three classification models which we will train with different levels of encoding and preprocessing of the dataset and by adjusting hyperparameters. With this method, we want to compare different models as well as different prerequisites within each model.

These are the classifier model we want to train:
* k-Nearest Neighbors: KNeighborsClassifier
* Support Vector Machine Algorithm (SVM): Linear Support Vector Classification (Linear SVC)
* Random Forest Classifier

## Splitting the dataset
Before training any models, the dataset has to be split into training and test dataset. We will use the standard 80-20 approach.

But before this can be done, the dataset needs another round of special attention. As 'patient_nbr' indicates (as shown in the previous notebook), one patient can appear as multiple encounters in this dataset, recognizable by 'patient_nbr'. We don't want encounters of the same patient appear in the training dataset as well as in the test dataset because even when dropping the column 'patient_nbr', it could distort the results.

There are two approaches which we will test and compare:
* having only one encounter for each patient, therefore dropping all lines of reappearing patient numbers (this means losing information)
* having a patient appear only in one part of the split (this means sorting the dataset by 'patient_nbr' before splitting, then randomizing the order again and dropping 'patient_nbr' after the split)

We will create methods for both approaches to be able to compare the effect when training different models.

In [62]:
unique_patients = len(data["patient_nbr"].unique())
number_encounters = len(data)
print(f"Number of unique patients: {unique_patients}")
print(f"Number of encounters: {number_encounters}")
print(f"Number of encounters in which a patient reappears in the dataset: {number_encounters-unique_patients}")

Number of unique patients: 71506
Number of encounters: 101742
Number of encounters in which a patient reappears in the dataset: 30236


Roughly 30% of the current state of the dataset are reappearing patients. Therefore, when dropping reappearing patient encounters, we will lose 30% of the data.

In [95]:
patient_encounters = data.groupby('patient_nbr').size()
max_encounters = patient_encounters.max()
patients_with_max_encounters = patient_encounters[patient_encounters == max_encounters].index.tolist()
avg_encounters = patient_encounters.mean()
median_encounters = patient_encounters.median()
single_encounter = (patient_encounters == 1).sum()

print(f"Maximum encounters of a patient: {max_encounters}")
print(f"Patient with {max_encounters} encounters: {patients_with_max_encounters}")
print(f"Average encounters of a patient: {avg_encounters}")
print(f"Median encounters of a patient: {median_encounters}")
print(f"Number of patients with one encounter: {single_encounter}")

Maximum encounters of a patient: 40
Patient with 40 encounters: [88785891]
Average encounters of a patient: 1.4228456353313008
Median encounters of a patient: 1.0
Number of patients with one encounter: 54739


In [64]:
patient_encounters

patient_nbr
135          2
378          1
729          1
774          1
927          1
            ..
189351095    1
189365864    1
189445127    1
189481478    1
189502619    1
Length: 71506, dtype: int64

The average number of encounters is 1.4. Therefore, the average patients appears more than once in this dataset.

In [65]:
data.loc[data["patient_nbr"] == patients_with_max_encounters[0]]

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,diag_1_Circulatory,diag_1_Diabetes,diag_1_Digestive,diag_1_Genitourinary,diag_1_Injury,diag_1_Musculoskeletal,diag_1_Neoplasms,diag_1_Other,diag_1_Respiratory,diag_2_Circulatory,diag_2_Diabetes,diag_2_Digestive,diag_2_Genitourinary,diag_2_Injury,diag_2_Missing,diag_2_Musculoskeletal,diag_2_Neoplasms,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Genitourinary,diag_3_Injury,diag_3_Missing,diag_3_Musculoskeletal,diag_3_Neoplasms,diag_3_Other,diag_3_Respiratory,admission_type_Elective,admission_type_Emergency,admission_type_NaN,admission_type_Newborn,admission_type_Trauma Center,admission_type_Urgent,metformin_bool,repaglinide_bool,nateglinide_bool,chlorpropamide_bool,glimepiride_bool,acetohexamide_bool,glipizide_bool,glyburide_bool,tolbutamide_bool,pioglitazone_bool,rosiglitazone_bool,acarbose_bool,miglitol_bool,troglitazone_bool,tolazamide_bool,insulin_bool,glyburide-metformin_bool,glipizide-metformin_bool,glimepiride-pioglitazone_bool,metformin-rosiglitazone_bool,metformin-pioglitazone_bool,age_label
23809,88785891,1,1,1,7,1,32,0,10,0,0,0,3,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
24669,88785891,1,1,1,7,1,10,0,10,1,0,2,2,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
24842,88785891,1,1,1,7,5,35,0,15,1,0,3,6,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
27017,88785891,1,1,1,7,2,32,0,4,1,0,6,9,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
29756,88785891,1,1,1,7,4,64,0,19,4,1,9,6,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
35051,88785891,1,1,1,7,8,75,1,22,3,2,12,9,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
36388,88785891,1,1,1,7,3,36,0,15,3,3,11,9,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
37916,88785891,1,1,1,7,2,41,0,12,2,2,11,4,0,3,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
41285,88785891,1,1,7,7,2,54,0,18,0,4,7,6,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2
45095,88785891,1,1,1,7,7,74,0,24,0,4,9,9,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2


#### Approach 1: Dropping reappearing patients

In [104]:
len(data["patient_nbr"].unique())

71506

**Method 'get_split_for_unique_patients'**

In [109]:
def get_split_for_unique_patients(data):
  data_unique_patients = data.drop_duplicates(subset='patient_nbr', keep="first")
  data_unique_patients = data_unique_patients.drop("patient_nbr", axis=1)
  features = data_unique_patients.columns[data_unique_patients.columns != "readmitted"].tolist()
  target_value = ["readmitted"]

  X = data_unique_patients[features]
  y = data_unique_patients[target_value]

  return train_test_split(X, y, test_size=0.2)

In [110]:
# example how to use the function to get the 80-20 split
X_train, X_test, y_train, y_test = get_split_for_unique_patients(data)

In [111]:
len(X_train) + len(X_test)

71506

### Approach 2: Sorting, splitting and mixing
Steps:
1. Sort dataset by 'patient_nbr'
2. Split dataset
3. Ensure that no patient from training appears in test
4. Mix both datasets (otherwise it will be ordered)

One problem that has to be addressed: It could be analyzed whether having a highly recurring patient in one dataset has a negative impact on the result.

**Method 'get_split_for_sorted_patients'**

In [123]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import shuffle

def get_split_for_sorted_patients(data):
  # define data set and target value
  X = data.drop(columns=["readmitted"])
  y = data["readmitted"]

  # define group
  groups = data["patient_nbr"]

  # instance of GroupShuffleSplit
  gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

  # Split dataset
  for train_idx, test_idx in gss.split(X, y, groups=groups):
      X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
      y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

  # check whether patient_nbr appear in both datasets
  common_patient_nbrs = set(X_train["patient_nbr"]).intersection(set(X_test["patient_nbr"]))
  print("common patient numbers:", common_patient_nbrs)

  # Remove patient_nbr column from both datasets
  X_train = X_train.drop(columns=["patient_nbr"])
  X_test = X_test.drop(columns=["patient_nbr"])

  # shuffle all datasets before returning than (using random_state to ensure that rows still match after shuffling)
  X_train = shuffle(X_train, random_state=42)
  X_test = shuffle(X_test, random_state=42)
  y_train = shuffle(y_train, random_state=42)
  y_test = shuffle(y_test, random_state=42)

  if len(common_patient_nbrs) == 0:
    return X_train, X_test, y_train, y_test


In [124]:
# example how to use the function to get the 80-20 split
X_train, X_test, y_train, y_test = get_split_for_sorted_patients(data)

common patient numbers: set()


In [129]:
y_train

78031    0
23755    0
94922    0
15067    0
6777     0
        ..
7812     0
68671    1
95899    0
1060     0
19702    1
Name: readmitted, Length: 81490, dtype: int64

In [130]:
X_train

,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,diag_1_Circulatory,diag_1_Diabetes,diag_1_Digestive,diag_1_Genitourinary,diag_1_Injury,diag_1_Musculoskeletal,diag_1_Neoplasms,diag_1_Other,diag_1_Respiratory,diag_2_Circulatory,diag_2_Diabetes,diag_2_Digestive,diag_2_Genitourinary,diag_2_Injury,diag_2_Missing,diag_2_Musculoskeletal,diag_2_Neoplasms,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Genitourinary,diag_3_Injury,diag_3_Missing,diag_3_Musculoskeletal,diag_3_Neoplasms,diag_3_Other,diag_3_Respiratory,admission_type_Elective,admission_type_Emergency,admission_type_NaN,admission_type_Newborn,admission_type_Trauma Center,admission_type_Urgent,metformin_bool,repaglinide_bool,nateglinide_bool,chlorpropamide_bool,glimepiride_bool,acetohexamide_bool,glipizide_bool,glyburide_bool,tolbutamide_bool,pioglitazone_bool,rosiglitazone_bool,acarbose_bool,miglitol_bool,troglitazone_bool,tolazamide_bool,insulin_bool,glyburide-metformin_bool,glipizide-metformin_bool,glimepiride-pioglitazone_bool,metformin-rosiglitazone_bool,metformin-pioglitazone_bool,age_label
78031,1,3,1,1,6,61,2,20,0,0,0,5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6
23755,1,1,3,5,6,43,1,20,0,0,0,6,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5
94922,1,3,1,1,3,26,6,12,0,0,1,7,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5
15067,0,1,6,7,6,58,3,19,0,0,0,5,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
6777,0,1,7,7,10,69,3,23,0,0,2,9,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7812,0,1,6,7,4,46,0,8,0,0,0,8,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6
68671,0,2,3,1,9,50,1,15,0,0,0,7,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6
95899,1,3,6,1,3,9,2,17,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
1060,0,6,1,17,1,35,1,8,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
